# solving LP primal

In [ ]:
using Random
using Ipopt
using MathOptInterface

const MOI = MathOptInterface;

In [ ]:
N = 10
D = 5;

## create a non-trivial LP problem

In [ ]:
s = rand(N)
s = 2*s.-1
λ = max.(-s, 0)
s = max.(s, 0)
x̂ = rand(D)
A = rand(N, D)
b = A*x̂ .+ s
c = -A'*λ;

In [ ]:
model = Ipopt.Optimizer()
x = MOI.add_variables(model, D)

# define objective
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

In [ ]:
# set constraints
for i in 1:N
    MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(A[i], x), -b[i]),MOI.LessThan(0.))
end

for i in 1:D
    MOI.add_constraint(model,MOI.SingleVariable(x[i]),MOI.LessThan(1.))
    MOI.add_constraint(model,MOI.SingleVariable(x[i]),MOI.GreaterThan(0.))
end

In [ ]:
MOI.optimize!(model)

In [ ]:
@assert MOI.get(model, MOI.TerminationStatus()) in [MOI.LOCALLY_SOLVED, MOI.OPTIMAL]

In [ ]:
x̄ = MOI.get(model, MOI.VariablePrimal(), x)